# Introduction
Under the Affordable Care Act, California runs its own health insurance marketplace known as Covered California. Although most residents of the state are insured through their employer or Medicare/Medicaid, the state-based marketplace is important [for insuring over 1.7 million Californians](https://www.cms.gov/newsroom/fact-sheets/marketplace-2023-open-enrollment-period-report-final-national-snapshot).

A key factor in choosing a health plan is cost. While a consumer's actual premium will vary based on individual factors, such as age and eligibility for advanced premium tax credits, we may be able to infer affordability by looking at the base rates of different plans. Covered California publishes [public data](https://hbex.coveredca.com/data-research/) for research, which will be used in this analysis. In addition to comparing rates between different plans, we'll also compare rates between the years 2023 and 2024 to see how they changed.

# Preparations for analysis

In [1]:
# import necessary Python libraries

import pandas as pd

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import sqlite3 as sql

print("Setup Complete")

Setup Complete


In [ ]:
# connect to SQL database

db = "/kaggle/input/covered-california-2023-2024/Covered California.db"
connection = sql.connect(db) # create connection object
print("Database connected")
table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", connection)
print(table)

# Data overview
First, a quick overview of the data:

In [ ]:
rates_2024_df = pd.read_csv("/kaggle/input/covered-california-2023-2024/Rates_2023.csv")
rates_2024_df.head()

The rates file shows us that within each plan, Individual Rate varies based on Age, Rating Area ID, Metal Level, and Network. The Rating Area ID is the geographic region that health plans serve.

In [ ]:
regions_2024_df = pd.read_csv("/kaggle/input/covered-california-2023-2024/Regions_2023.csv")
regions_2024_df.tail()

As we see here, there are 19 different rating regions in California, which roughly correspond to county. Some regions include multiple counties. From a business perspective, we'd focus primarily on the local community that we serve, so we'll pick just one area to analyze for now. Let's focus on Los Angeles County.

In [ ]:
query = """
        SELECT DISTINCT
            CoveredCaliforniaRatingRegion
        FROM
            Regions_2024
        WHERE
            County = "LOS ANGELES" -- Note that SQLite is case-sensitive. In this case, we know all "Los Angeles" values are in all caps.
        """
pd.read_sql(query, connection)

LA spans two different rating regions: 15 and 16.

In [ ]:
query = """
        SELECT
            CoveredCaliforniaRatingRegion
            , ZipCode
        FROM
            Regions_2024
        WHERE
            County = "LOS ANGELES"
        """
la_zip = pd.read_sql(query, connection)
la_zip

Cross-refererencing these results with a [zip code map](https://data.lacounty.gov/datasets/lacounty::la-county-zip-codes/explore?location=33.795159%2C-118.298800%2C7.59), we see that region 15 corresponds to the eastern (and northernmost) sections of LA, and region 16 corresponds to the western section. We'll refer to these as East LA County and West LA County, respectively.

# Cost analysis
To keep comparisons fair, we'll focus on the silver tier of each health plan. It is one of the only tiers that every health plan offers (the others being gold and platinum), and it is by far the most popular selection year after year. Enrollees at the silver level (enhanced or otherwise) tend to make up at least 50% of all buyers (see [Profile Selection data](https://hbex.coveredca.com/data-research/)), so they represent a core segment of the market.

In [ ]:
# find the average silver rate across all of LA County
query = """
        SELECT
            RatingAreaID
            , ROUND(AVG(IndividualRate), 2) AS AverageRate
            , Applicant
            , MetalLevel
            , Network
        FROM
            Rates_2024
        WHERE
            (RatingAreaID = "Rating Area 15" OR RatingAreaID = "Rating Area 16")
            AND
            MetalLevel = "Silver"
        GROUP BY
            RatingAreaID
            , Applicant
            , MetalLevel
            , Network
        """
silver_data = pd.read_sql(query, connection)
silver_data

In [ ]:
# set style and title
plt.figure(figsize=(14,7))
plt.title("Figure 1: Average Silver Rate of Each Health Plan")
sns.set_style("darkgrid")

# bar chart showing average silver rates of each health plan
sns.barplot(data=silver_data, x="AverageRate", y="Applicant", hue="Network", errorbar=("ci", 0), dodge=False)

Blue Shield PPO is the most expensive option at the silver level. Although higher costs are to be expected of PPOs due to greater flexibility, it surpasses the only other PPO in the county by about \\$100. It is also nearly twice as expensive as L.A. Care, which is the least expensive option.

In [ ]:
# breakdown of costs at all metal levels
query = """
        SELECT
            RatingAreaID
            , ROUND(AVG(IndividualRate), 2) AS AverageRate
            , Applicant
            , MetalLevel
            , Network
        FROM
            Rates_2024
        WHERE
            (RatingAreaID = "Rating Area 15" OR RatingAreaID = "Rating Area 16")
        GROUP BY
            RatingAreaID
            , Applicant
            , MetalLevel
            , Network
        """
allmetal_data = pd.read_sql(query, connection)
allmetal_data

In [ ]:
# set style and title
plt.figure(figsize=(14,7))
plt.title("Figure 2: Average Rates of All Metal Levels by Health Plan")
sns.set_style("darkgrid")

# bar chart showing average rates within each health plan, with a color split by metal level
sns.barplot(data=allmetal_data, x="AverageRate", y="Applicant", hue="MetalLevel", palette="BrBG_r",
            errorbar=("ci", 0), hue_order=["Platinum", "Gold", "Silver", "Bronze", "Catastrophic", "HDHP"])

Here, it is easier to see that not all health plans offer the same metal tiers. Notably, the difference between platinum and gold is especially high for Anthem, Blue Shield PPO, or Health Net PPO.

When shopping around, you likely already have a metal level in mind based on your own finances and medical needs, so let's rearrange the plot to focus on metal level.

In [ ]:
# set style and title
plt.figure(figsize=(14,7))
plt.title("Figure 3: Average Rates by Metal Level")
sns.set_style("darkgrid")

# bar chart showing average rates for each metal level, with a color split by health plan
sns.barplot(data=allmetal_data, x="AverageRate", y="MetalLevel", hue="Applicant", palette="terrain",
            errorbar=("ci", 0), order=["Platinum", "Gold", "Silver", "Bronze", "Catastrophic", "HDHP"])

Again, Blue Shield PPO tops the chart heavily across all levels. The other PPO, which is offered by Health Net, has rates that are more competitive with HMO plans at the bronze, catastrophic, and high deductible health plan (HDHP) levels. Anthem HMO is one of the more expensive options by far if you opt for platinum, but it is roughly in line with other plans at lower tiers. L.A. Care is still one of the most affordable options at all commonly available levels.

In [ ]:
# set style and title
plt.figure(figsize=(14,7))
plt.title("Figure 4: Average Silver Rate per Rating Region")
sns.set_style("darkgrid")

# bar chart showing average silver rates of each health plan within respective rating regions
sns.barplot(data=silver_data, x="AverageRate", y="Applicant", hue="RatingAreaID", errorbar=("ci", 0), palette="Set1")

Rates in West LA County are higher across the board. The gaps are particularly large for the PPO plans, with about a \\$100 difference for Blue Shield and about a \\$200 difference for Health Net.

In [ ]:
# add a column for year to the rates tables and combine the tables so that we can compare them
# again, we'll only look at silver to keep the comparisons fair
query = """
        SELECT
        /* use IIF to attach the year using any criterion that all rows meet
        in this case, rows where RatingAreaID does not have null values */
            IIF(RatingAreaID NOT NULL, 2023, "error") AS Year
            , RatingAreaID
            , ROUND(AVG(IndividualRate), 2) AS AverageRate
            , Applicant
            , MetalLevel
            , Network
        FROM
            Rates_2023
        WHERE
            (RatingAreaID = "Rating Area 15" OR RatingAreaID = "Rating Area 16")
            AND
            MetalLevel = "Silver"
        GROUP BY Year
            , RatingAreaID
            , Applicant
            , MetalLevel
            , Network
            
        UNION

        SELECT
            IIF(RatingAreaID NOT NULL, 2024, "error") AS Year
            , RatingAreaID
            , ROUND(AVG(IndividualRate), 2) AS AverageRate
            , Applicant
            , MetalLevel
            , Network
        FROM
            Rates_2024
        WHERE
            (RatingAreaID = "Rating Area 15" OR RatingAreaID = "Rating Area 16")
            AND
            MetalLevel = "Silver"
        GROUP BY
            Year
            , RatingAreaID
            , Applicant
            , MetalLevel
            , Network
            
        ORDER BY Applicant
        """
yearly_data = pd.read_sql(query, connection)
yearly_data

In [ ]:
# set style and title
plt.figure(figsize=(14,7))
plt.title("Figure 5: Yearly Average Rate for Each Health Plan")
sns.set_style("darkgrid")

# bar chart showing average silver rates of each health plan per year
sns.barplot(data=yearly_data, x="AverageRate", y="Applicant", hue="Year", errorbar=("ci", 0), palette="prism")

Rates increased all around from the previous year. Blue Shield PPO increased by about \\$100, which is a much bigger difference than other plans had. On the other hand, L.A. Care, Molina, and Kaiser kept their rates fairly similar between years. Of note, Oscar left the California market in 2024.

# Final thoughts
I hope this analysis provided some insight into the costs of health insurance, as well as inspiration for further exploration. It is important to remember that there are other factors that determine the final cost to the consumer, as well as factors that determine how insurance companies calculate base rates to ensure a balance of profitability and affordability. Future studies might be done to determine the strength of the correlations between these factors and insurance rates.